# Data collection: 1st try

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

page = requests.get('http://live.chinaz.com/anchor.html?platform=all&gameType=all&timeType=thismonth&date=20180325&method=power')
soup = BeautifulSoup(page.text, 'html.parser')

# Name & Platform
artist_name_list = soup.find_all(class_='info_avator')
artist_name = []
artist_platform = []

for i in range(len(artist_name_list)):
    artist_name.append(artist_name_list[i].find('span').contents[0])
    artist_platform.append(artist_name_list[i].find('i').contents[0])

    
# Classification
artist_classification_list = soup.find_all(class_='info_platform')
artist_classification = []

for i in range(len(artist_classification_list)):
    artist_classification.append(artist_classification_list[i].find('span').contents[0])


# URL in live.chinaz.com
url_livechinaz_list = soup.find_all(class_='info')
url_livechinaz = []

for i in range(len(url_livechinaz_list)):
    url_livechinaz.append(url_livechinaz_list[i].find('a').get('href'))
    
    
url_artist = []
for i in range(len(url_livechinaz)):
    try:
        page2 = requests.get(url_livechinaz[i])
        soup2 = BeautifulSoup(page2.text, 'html.parser')

        # URL of the artist
        url_artist_list = soup2.find(class_='video clear')
        url_artist.append(url_artist_list.find('a').get('href')[:-16])
    except requests.Timeout as err:
        url_artist.append(err.message)
        
        
# CSV
with open('APMA4990_TEST.csv','w',newline='',encoding='utf-8-sig') as f:
    w = csv.writer(f)
    w.writerow(['名字', '平台','类别','数据库地址'])
    for i in range(50):
        w.writerow([artist_name[i], artist_platform[i], artist_classification[i], url_artist[i]])


1. problem with time delay
2. can only get first 50 streamer

reference:
[1] https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3 
[2] https://www.cnblogs.com/xinyangsdut/p/7617691.html

# 2nd try 

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
from selenium import webdriver
import pandas as pd

# 使用Chrome浏览器创建浏览器对象
options = webdriver.ChromeOptions()
options.add_argument('--headless')

# Initialize the driver
driver = webdriver.Chrome(chrome_options=options)

# 使用get方法加载页面
driver.get("http://live.chinaz.com/anchor.html?platform=all&gameType=all&timeType=thismonth&date=20180325&method=power")

# 展开页面，提取n个数据
for i in range(1,8):
    # class="zhubo_btn"是下一页按钮，click() 是模拟点击
    driver.find_element_by_class_name('zhubo_btn').click()
    # 给3s响应时间
    time.sleep(5)
    
soup = BeautifulSoup(driver.page_source, "lxml")

# Name & Platform
artist_name_list = soup.find_all(class_='info_avator')
artist_name = []
artist_platform = []

#从info_avator抓出了一堆文字，挑选span里面的是主播名，i是平台名
for m in range(len(artist_name_list)):
    try:
        artist_name.append(artist_name_list[m].find('span').contents[0])
        artist_platform.append(artist_name_list[m].find('i').contents[0])
    except:
        print(m+1) # 出错数据的排名
        artist_name.append('Error Name')
        artist_platform.append('Error Platform')
        
# Rank
artist_rank_list = soup.find_all(class_='rank')
artist_rank = []

for m in range(len(artist_rank_list)):
    artist_rank.append(artist_rank_list[m].find('div').contents[0])

for m in range(50):
    artist_rank[m] = str(m+1)
    
# Classification
artist_classification_list = soup.find_all(class_='info_platform')
artist_classification = []

for i in range(len(artist_classification_list)):
    try:
        artist_classification.append(artist_classification_list[i].find('span').contents[0])
    except:
        print(i+1) # 出错数据的排名
        artist_classification.append('Error Classification')
        
# URL in live.chinaz.com
url_livechinaz_list = soup.find_all(class_='info')
url_livechinaz = []

for i in range(len(url_livechinaz_list)):
    try:
        url_livechinaz.append(url_livechinaz_list[i].find('a').get('href'))
    except:
        print(i+1) # 出错数据的排名
        url_livechinaz.append('Error URL in live.chinaz.com')
        
# CSV
with open('APMA4990_TEST_500.csv','w',newline='',encoding='utf-8-sig') as f:
    w = csv.writer(f)
    w.writerow(['排名', '名字', '平台','类别','数据库地址'])
    for i in range(len(url_livechinaz)):
        w.writerow([artist_rank[i],artist_name[i], artist_platform[i], artist_classification[i], url_livechinaz[i]])


1. ChromeDriver,get 400 data 
2. No.392: timeout error 
3. add rank to quickly check 
4. time.sleep is important! or will result in problem of showing

Reference 
[1] https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3 
[2] https://www.cnblogs.com/xinyangsdut/p/7617691.html 
[3] https://intoli.com/blog/running-selenium-with-headless-chrome/ 
[4] https://developers.google.com/web/updates/2017/04/headless-chrome

# change the url

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
from selenium import webdriver
import pandas as pd

df = pd.read_csv('C:\\Users\Owner\Python\APMA4990_TEST_1950.csv',index_col=None)
df.head()

# URL of the artist
url_artist = []

for i in range(1950):
    try:
        page = requests.get(df.loc[i,'数据库地址'])
        soup = BeautifulSoup(page.text, 'html.parser')
        url_artist_list = soup.find(class_='video clear')
        url_artist.append(url_artist_list.find('a').get('href')[:-16])
    except:
        url_artist.append('Error')
        
# CSV
with open('APMA4990_TEST_4_9.csv','w',newline='',encoding='utf-8-sig') as f:
    w = csv.writer(f)
    w.writerow(['排名', '主播', '平台','类别','直播间地址'])
    for i in range(len(url_artist)):
        w.writerow([df.loc[i,'排名'],df.loc[i,'名字'], df.loc[i,'平台'], df.loc[i,'类别'], url_artist[i]])